# Risky Moon - Problem 353
<p>
A moon could be described by the sphere $C(r)$ with centre $(0,0,0)$ and radius $r$. 
</p>

<p>
There are stations on the moon at the points on the surface of $C(r)$ with integer coordinates. The station at $(0,0,r)$ is called North Pole station, the station at $(0,0,-r)$ is called South Pole station.
</p>

<p>
All stations are connected with each other via the shortest road on the great arc through the stations. A journey between two stations is risky. If <var>d</var> is the length of the road between two stations, $\left(\frac{d}{\pi r}\right)^2$ is a measure for the risk of the journey (let us call it the risk of the road). If the journey includes more than two stations, the risk of the journey is the sum of risks of the used roads.
</p>

<p>
A direct journey from  the North Pole station to the South Pole station has the length $\pi r$ and risk $1$. The journey from the North Pole station to the South Pole station via $(0,r,0)$ has the same length, but a smaller risk:</p>
\[
\left(\frac{\frac{1}{2}\pi r}{\pi r}\right)^2+\left(\frac{\frac{1}{2}\pi r}{\pi r}\right)^2=0.5
\]

<p>
The minimal risk of a journey from the North Pole station to the South Pole station on $C(r)$ is $M(r)$.
</p>

<p>
You are given that $M(7)=0.1784943998$ rounded to $10$ digits behind the decimal point. 
</p>

<p>
Find $\displaystyle{\sum_{n=1}^{15}M(2^n-1)}$.
</p>

<p>
Give your answer rounded to $10$ digits behind the decimal point in the form a.bcdefghijk.
</p>



## Solution.
We rephrase the problem and go from $(-r, 0, 0)$ to $(r, 0 ,0)$.

Ideas:
- Dijisktra
- connect only to the closest points

In [251]:
from math import acos, pi, sqrt, ceil, gcd
from collections import defaultdict
import bisect

In [252]:
def risk(a, b, r):
    '''
    Risk of the journey between a=(a0, a1, a2) and b=(b0, b1, b2) divided by pi.
    '''
    d = sum((i-j)**2 for i, j in zip(a, b))
    return acos(1 - 1/2 * d/r**2) ** 2 / pi**2   

In [72]:
def is_sq(x):
    '''
    Checks if x is a perfect square.
    '''
    sq = int(sqrt(x))
    return sq * sq == x

def generate_fundamental_triples(r):
    '''
    Generates all possible integer (x, y, z) s.t. x,y,z >= 0, x >= y >= z and
    x^2 + y^2 + z^2 = r^2.
    '''
    triples = []
    for x in range(r, int(r/sqrt(3)) - 1, -1):
        for y in range(int(sqrt(r**2-x**2)), ceil(sqrt((r**2 - x**2)/2))-1, -1):
            if is_sq(r**2 - x**2 - y**2):
                triples.append((x, y, int(sqrt(r**2 - x**2 - y**2))))
    return triples

In [204]:
def generate_graph(r):
    '''
    V - set of vertices
    E - dictionary of neighbours
    '''
    V = set()
    E = {}

    triples = generate_fundamental_triples(r)
    levels = defaultdict(set)

    for triple in triples:
        x,y,z = triple
        levels[x].add((x,y,z))
        levels[x].add((x,z,y))
        levels[y].add((y,x,z))
        levels[y].add((y,z,x))
        levels[z].add((z,x,y))
        levels[z].add((z,y,x))
        levels[-x].add((-x,y,z))
        levels[-x].add((-x,z,y))
        levels[-y].add((-y,x,z))
        levels[-y].add((-y,z,x))
        levels[-z].add((-z,x,y))
        levels[-z].add((-z,y,x))

    levels = dict(sorted(levels.items()))

    # create vertices
    V = set([v for x in levels.values() for v in x])


    # create edges
    depth = 100 # can be changed
    l = len(levels)
    keys = list(levels.keys())
    E = defaultdict(set)

    for i, lvl in enumerate(keys):
        for v in levels[lvl]:
            for j in range(max(0, i - depth), min(l, i + depth)):
                for u in levels[keys[j]]:
                    E[v].add(u)        

    
    return V, E

In [191]:
def dijkstra(graph, r, start, end):
    '''
    https://web.engr.oregonstate.edu/~glencora/wiki/uploads/dijkstra-proof.pdf
    '''
    V = graph[0]
    E = graph[1]
    d = {v:float('inf') for v in V}
    d[start] = 0
    
    R = set()
    l = len(V)

    while len(R) != l:
        # pick u in V\R with the smallest d
        current_min = float('inf')
        for w in V - R:
            if d[w] < current_min:
                current_min = d[w]
                u = w

        # u is now seen
        R.add(u)

        # update neighbours of u
        for w in E[u]:
            new_d = d[u] + risk(u, w, r)
            if d[w] > new_d:
                d[w] = new_d

        # if we visit end, we can just return the total risk as it's additive
        if end in R:
            return d[end]

In [192]:
def M(r):
    return dijkstra(generate_graph(r), r, (r, 0, 0), (-r, 0, 0))

In [205]:
for n in range(1, 16, 1):
    M_values[n] = M(2**n - 1)
    print(n, 2**n - 1, M_values[n])

1 1 0.5
2 3 0.2360456664461351
3 7 0.1784943997551485
4 15 0.10714794796211999
5 31 0.08744530034378603
6 63 0.041515684149188534
7 127 0.046342185231883465
8 255 0.023951558104512362
9 511 0.017904889542828168
10 1023 0.010340579258887034
11 2047 0.010981092024029224
12 4095 0.0050289498357206155
13 8191 0.005600730815026471
14 16383 0.0029262924103760444
15 32767 0.002260757199644657


In [278]:
s = 0
for v in M_values.values():
    s += v
    print(v)
print(round(s, 10)) #1.2759860331

0.2360456664461351
0.1784943997551485
0.10714794796211999
0.08744530034378603
0.041515684149188534
0.046342185231883465
0.023951558104512362
0.017904889542828168
0.010340579258887034
0.010981092024029224
0.0050289498357206155
0.005600730815026471
0.0029262924103760444
0.002260757199644657
0.5
1.2759860331
